In [2]:
import numpy as np
from numpy import array
from sklearn.feature_extraction.text import TfidfVectorizer

test_x_path = './test.dat'
test_y_path = './format.dat'
train_x_path = './train.dat'

x_train, y_train, x_test, y_test = [], [], [], []

with open(train_x_path, 'r') as f:
    for line in f:
        i, j = line.split('\t')
        y_train.append(int(i))
        x_train.append(j)

with open(test_x_path, 'r') as f:
    for line in f:
        x_test.append(line)
        
with open(test_y_path, 'r') as f:
    for line in f:
        y_test.append(int(line))

x_train = array(x_train)
y_train = array(y_train)
x_test = array(x_test)
y_test = array(y_test)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [34]:

xtrain, xtest, ytrain, ytest = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)
# from gensim.utils import simple_preprocess

vectorizer = TfidfVectorizer(token_pattern = r'(?u)\b\w\w\w\w+\b')

xtrain = vectorizer.fit_transform(xtrain)
xtest = vectorizer.transform(xtest)
for k in range(15, 22, 1):
    cossims = xtest.dot(xtrain.T).toarray()
    ans = []
    for m, csms in enumerate(cossims):
        sims = array([(csms[i], int(i)) for i in range(len(csms))], dtype=object)
        sims = array(sorted(sims, key=lambda x: x[0], reverse=True)[:k])
        ans_arr = array([0.0]*6)
        for i, j in sims:
            ans_arr[ytrain[j]] += (1/(1.001-i)) if i > 0.1 else 0
        ans.append(ans_arr.argmax())
        if m%1000==1:
            print(m)
    print(k, f1_score(ytest, ans, average='macro'))

1
1001
2001
15 0.4716117776357396
1
1001
2001
16 0.47406725026090746
1
1001
2001
17 0.47119936930115164
1
1001
2001
18 0.47025381665010224
1
1001
2001
19 0.4709332692808793
1
1001
2001
20 0.4691877154827499
1
1001
2001
21 0.4676714781038446


In [ ]:
f1_score(ytest, ans, average='macro')

0.466690098229804

In [2]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(x_train)
X.shape

(14438, 38060)

In [3]:
X_test = vectorizer.transform(x_test)

In [4]:
X_test.shape

(14442, 38060)

In [5]:
cossim_mat = X_test.dot(X.T)

In [6]:
cossim_mat.shape
# (test, train)

(14442, 14438)

In [7]:
cossim_arr = cossim_mat.toarray()
cossim_arr.shape
#test, train

(14442, 14438)

In [8]:
print(cossim_arr)

[[0.04468925 0.06735867 0.03636464 ... 0.06460621 0.08486988 0.0656811 ]
 [0.02648307 0.03843503 0.01928721 ... 0.0481624  0.02906032 0.03076613]
 [0.01796505 0.04231999 0.01849211 ... 0.02514022 0.12432123 0.02687808]
 ...
 [0.02451196 0.03474795 0.01558273 ... 0.03576132 0.03022565 0.03302707]
 [0.00873286 0.03414532 0.00684567 ... 0.01204491 0.01869213 0.0363827 ]
 [0.028238   0.04873663 0.01830453 ... 0.03866297 0.03201441 0.06096187]]


In [18]:
ans = []
for m, csms in enumerate(cossim_arr):
    sims = array([(csms[i], int(i)) for i in range(len(csms))], dtype=object)
#     print(m, sims)
    sims = array(sorted(sims, key=lambda x: x[0], reverse=True)[:k])
    ans_arr = array([0.0]*6)
    for i, j in sims:
        ans_arr[y_train[j]] += (1/(1.001-i)) if i > 0.1 else 0
    ans.append(ans_arr.argmax())
    if m%1000==1:
        print(m)


1
1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
11001
12001
13001
14001


In [19]:
sum([
    i == j for i,j in zip(ans, y_test)
])/len(y_test)

0.23542445644647556

In [21]:
with open('output1.txt', 'w') as f:
    for i in ans:
        f.write(str(i)+'\n')
        